## Import Statements

In [2]:
import precovis_v001 as precovis

import time
import numpy as np
import pandas as pd
import sqlite3 as sql
from astropy.time import Time
from astroquery.jplsbdb import SBDB
#import healpy as hp

ModuleNotFoundError: No module named 'precovery'

In [ ]:
import importlib
importlib.reload(precovis)

## Current Development
### Currently working on: Dropdown Menus

In [ ]:
start_time = time.time()

#Curdoc Clear and Creation
asteroid_doc = precovis.curdoc()
asteroid_doc.clear()

#Entire Date Range: 58270,59972
#One Line Date Range: 58582, 58920
start = 58270
end = 59972
dur = 0.5

coords_df, transitions_orbit = precovis.objectToCoordsDF("2020 AV2", start, end, True)

#Plot Set-up (Main)
orbit_plot = precovis.plotSetUp("Asteroid Orbit Plot", 400, 600)

#Healpixel Set-up & Plotting (Main)
patches, merged_healpixels = precovis.plotIntercectingHealpixels(orbit_plot, "2020 AV2", start, end, dur, 'I41', 'ztf')

#Orbit Set-up & Plotting (Main)
orbit_lines = precovis.plotOrbitPaths(orbit_plot, "2020 AV2", start, end)



#Plot Set-up (Sub)
observation_plot = precovis.plotSetUp("Subsection Observation Plot", 400, 400)
observation_plot.height = 400
observation_plot.width = 400
patches, merged_healpixels = precovis.plotIntercectingHealpixels(observation_plot, "2020 AV2", start, end, dur, 'I41', 'ztf')


#Table Creation
data_table = precovis.tableCreation(merged_healpixels)

#Slider Creation
date_range_slider = precovis.mjdSliderCreation(start,end)

#Download Button Creation
download_button = precovis.Button(label="Download", button_type="default")

#White Space
dumdiv = precovis.Div(text='',width=10)

#White Space 2
dumdiv2 = precovis.Div(text='',width=10)

#Indicator Div for Patch Tapping
indicator_div = precovis.Div(text="no")




#Dropdown Creations
from bokeh.models import Select
obs_select = Select(value='I41', title='Observatory Selection', options=list(np.unique(precovis.frames['obscode'])))
dataset_id_select = Select(value='ztf', title='Dataset Selection', options=list(np.unique(precovis.frames['dataset_id'])))
#asteroid_select = Select(value='2020 AV2', title='Asteroid Selection', options=list()))

#Enter Button Creation
change_button = precovis.Button(label="Change Plot", button_type="default")



#Plot Updating (Main)
plot_callback = precovis.CustomJS(args=dict(all_sources=precovis.all_sources), code="""
    
    const closestIndex = (num, arr) => {
       let curr = arr[0], diff = Math.abs(num - curr);
       let index = 0;
       for (let val = 0; val < arr.length; val++) {
          let newdiff = Math.abs(num - arr[val]);
          if (newdiff < diff) {
             diff = newdiff;
             curr = arr[val];
             index = val;
          };
       };
       return index;
    };

    var data = [];
    var ref = [];
    for (var ii = 0; ii < all_sources.length; ii=ii+2){
        //console.log("ii"+ii)
        data[ii] = all_sources[ii].data;
        ref[ii] = all_sources[ii+1].data;
        

        const index_range = [closestIndex(cb_obj.value[0], ref[ii]["time"]), closestIndex(cb_obj.value[1], ref[ii]["time"])+1];
        
        const columns = Object.keys(data[ii])
        console.log(columns)
        //console.log("Before slider column lengths " + ii + ": " + data[ii]["ra"].length + ", " + data[ii]["dec"].length + ", " + data[ii]["healpixel"].length+ ", " + data[ii]["time"].length);

        for (var c = 0; c < columns.length; c++){
            data[ii][columns[c]] = ref[ii][columns[c]].slice(index_range[0], index_range[1]);
        }
        
        //data[ii]["index"] = ref[ii]["index"].slice(index_range[0], index_range[1]);
        //data[ii]["dec"] = ref[ii]["dec"].slice(index_range[0], index_range[1]);
        //data[ii]["ra"] = ref[ii]["ra"].slice(index_range[0], index_range[1]);
        //data[ii]["healpixel"] = ref[ii]["healpixel"].slice(index_range[0], index_range[1]);
        //data[ii]["time"] = ref[ii]["time"].slice(index_range[0], index_range[1]);
        
        //console.log("After slider column lengths " + ii + ": " + data[ii]["ra"].length + ", " + data[ii]["dec"].length + ", " + data[ii]["healpixel"].length+ ", " + data[ii]["time"].length);

        all_sources[ii].change.emit();
    };

    """)

  

table_callback = precovis.CustomJS(args=dict(source=precovis.table_source, ref_source=precovis.table_ref_source), code="""
   
    const closestIndex = (num, arr) => {
       let curr = arr[0], diff = Math.abs(num - curr);
       let index = 0;
       for (let val = 0; val < arr.length; val++) {
          let newdiff = Math.abs(num - arr[val]);
          if (newdiff < diff) {
             diff = newdiff;
             curr = arr[val];
             index = val;
          };
       };
       return index;
    };
    
    const data = source.data;
    const ref = ref_source.data;

    //console.log("Before slider column lengths: " + data["healpixel"].length + ", " + data["exposure_mjd_start"].length + ", " + data["exposure_mjd_mid"].length + ", " + data["exposure_duration"].length + ", " + data["id"].length + ", " + data["dataset_id"].length + ", " + data["data_uri"].length + ", " + data["obscode"].length + ", " + data["filter"].length);
    
    const index_range = [closestIndex(cb_obj.value[0], ref["exposure_mjd_mid"]), closestIndex(cb_obj.value[1], ref["exposure_mjd_mid"])+1];
    const columns = Object.keys(data)
    
    for (var c = 0; c < columns.length; c++){
        data[columns[c]] = ref[columns[c]].slice(index_range[0], index_range[1]);
    }
    
    //data["healpixel"] = ref["healpixel"].slice(index_range[0], index_range[1]);
    //data["exposure_mjd_start"] = ref["exposure_mjd_start"].slice(index_range[0], index_range[1]);
    //data["exposure_mjd_mid"] = ref["exposure_mjd_mid"].slice(index_range[0], index_range[1]);
    //data["exposure_duration"] = ref["exposure_duration"].slice(index_range[0], index_range[1]);
    //data["id"] = ref["id"].slice(index_range[0], index_range[1]);
    //data["dataset_id"] = ref["dataset_id"].slice(index_range[0], index_range[1]);
    //data["exposure_id"] = ref["exposure_id"].slice(index_range[0], index_range[1]);
    //data["data_uri"] = ref["data_uri"].slice(index_range[0], index_range[1]);
    //data["obscode"] = ref["obscode"].slice(index_range[0], index_range[1]);
    //data["filter"] = ref["filter"].slice(index_range[0], index_range[1]);
    //data["index"] = ref["index"].slice(index_range[0], index_range[1]);
    //data["filter"] = ref["filter"].slice(index_range[0], index_range[1]);
    //data["data_offset"] = ref["data_offset"].slice(index_range[0], index_range[1]);
    //data["data_length"] = ref["data_length"].slice(index_range[0], index_range[1]);
    //data["ra"] = ref["ra"].slice(index_range[0], index_range[1]);
    //data["dec"] = ref["dec"].slice(index_range[0], index_range[1]);
    //data["time"] = ref["time"].slice(index_range[0], index_range[1]);
    
    //console.log("After slider column lengths: " + data["healpixel"].length + ", " + data["exposure_mjd_start"].length + ", " + data["exposure_mjd_mid"].length + ", " + data["exposure_duration"].length + ", " + data["id"].length + ", " + data["dataset_id"].length + ", " + data["data_uri"].length + ", " + data["obscode"].length + ", " + data["filter"].length);
    
    source.change.emit();
    """)


mouse_click_callback = precovis.CustomJS(args=dict(obs_plot_xrange=observation_plot.x_range, obs_plot_yrange=observation_plot.y_range), code="""
    
    var event_x = cb_obj.x
    var event_y = cb_obj.y
    var event_sx = cb_obj.sx
    var event_sy = cb_obj.sy
    
    //console.log("Event x: " + event_x)
    //console.log("X Range Start:" + obs_plot_xrange.start)
    //console.log("X Range:" + obs_plot_xrange)
    
    obs_plot_xrange.start = event_x - 5
    obs_plot_xrange.end = event_x + 5
    obs_plot_yrange.start = event_y - 5
    obs_plot_yrange.end = event_y + 5
    
    obs_plot_xrange.change.emit()
    obs_plot_yrange.change.emit()
    """)


download_button_javaScript= """
        var data = source.data;
        var nrows = source.get_length();
        const columns = Object.keys(data);
        
        var out = "";
        
        for (var c = 0; c < columns.length-1; c++){
            out += columns[c] + ",";
        }
        out = out + columns[columns.length-1] + "\\n"
        
        for (var i = 0; i < nrows; i++) {
            for (var d = 0; d < columns.length-1; d++){
                out += data[columns[d]][i] + ",";
            }
            out += data[columns[columns.length-1]][i] + "\\n"; //This is where the error is
        }
        
        //var out = "healpixel, exposure_mjd_start, exposure_mjd_mjd, exposure_duration, id, dataset_id, exposure_id, data_uri, obscode, filter, data_offset, data_length, ra, dec, time\\n";
        //for (var i = 0; i < nrows; i++) {
        //    out += data['healpixel'][i] + "," + data['exposure_mjd_start'][i] + "," + data['exposure_mjd_mid'][i] + "," + data['exposure_duration'][i] + "," + data['id'][i] + "," + data['dataset_id'][i] + "," + data['exposure_id'][i] + "," + data['data_uri'][i] + "," + data['obscode'][i] + "," + data['filter'][i] +  + "," + data['data_offset'][i]  + "," + data['data_length'][i]  + "," + data['ra'][i]  + "," + data['dec'][i]  + "," + data['time'][i] + "\\n";
        //}
        
        var file = new Blob([out], {type: 'text/plain'});
        var elem = window.document.createElement('a');
        elem.href = window.URL.createObjectURL(file);
        elem.download = 'subsection_orbit_path_data.csv';
        document.body.appendChild(elem);
        elem.click();
        document.body.removeChild(elem);
        """


patch_indicator_callback = precovis.CustomJS(args=dict(patches=precovis.pixel_sources, div=indicator_div), code="""
console.log(cb_obj.indices)
console.log(cds.selected.indices)  
// note that these are the same. cds.selected is the object that this callback is set for,
// so you could use either one of these to refer to the data in your data source (below). 
// if the indices array is empty, user clicked somewhere not on a patch

for (var i = 0; i < pixel_sources.length; i++){
    if (cb_obj.indices.length == 0) {
        div.text = "no"
    }
    else {
        div.text = "yes: "+cds.data.name[cb_obj.indices]
    }
};
""")



orbit_plot.js_on_event(precovis.DoubleTap, mouse_click_callback)

date_range_slider.js_on_change('value_throttled', plot_callback)
date_range_slider.js_on_change('value_throttled', table_callback)

download_button.js_on_event(precovis.ButtonClick, precovis.CustomJS(args=dict(source=precovis.table_source), code=download_button_javaScript))

for pixel_source in precovis.pixel_sources:    
    pixel_source.selected.js_on_change('indices', patch_indicator_callback)




#Add everything to curdoc()
'''
row1 = precovis.Row(orbit_plot,observation_plot)
row2 = precovis.Row(date_range_slider, dumdiv, obs_select, dumdiv2, button)
layout = precovis.Column(row1, row2, data_table)
'''
select_col = precovis.Column(date_range_slider, precovis.Row(obs_select, dumdiv, dataset_id_select), change_button)
row1 = precovis.Row(orbit_plot,select_col)
layout = precovis.Column(row1, observation_plot, dumdiv2, data_table, download_button)
asteroid_doc.add_root(layout)

#precovis.save(asteroid_doc, "slider+table_v05.html")
#Don't touch slider+table.html since that is on the public drive now

end_time = time.time()
total_time = end_time - start_time
print("Time taken to run script:",total_time) #Seems to currently be around 120 seconds (half of them due to sorting dataset_id and obscode)

precovis.output_notebook()
precovis.show(layout)

## Working Version (Saved html file version)

In [ ]:
start_time = time.time()

#Curdoc Clear and Creation
asteroid_doc = precovis.curdoc()
asteroid_doc.clear()

#Entire Date Range: 58270,59972
#One Line Date Range: 58582, 58920
start = 58270
end = 59972
dur = 0.5

coords_df, transitions_orbit = precovis.objectToCoordsDF("2020 AV2", start, end, True)

#Plot Set-up (Main)
orbit_plot = precovis.plotSetUp("Asteroid Orbit Plot", 400, 600)

#Healpixel Set-up & Plotting (Main)
patches, merged_healpixels = precovis.plotIntercectingHealpixels(orbit_plot, "2020 AV2", start, end, dur, 'I41', 'ztf')

#Orbit Set-up & Plotting (Main)
orbit_lines = precovis.plotOrbitPaths(orbit_plot, "2020 AV2", start, end)



#Plot Set-up (Sub)
observation_plot = precovis.plotSetUp("Subsection Observation Plot", 400, 400)
observation_plot.height = 400
observation_plot.width = 400
patches, merged_healpixels = precovis.plotIntercectingHealpixels(observation_plot, "2020 AV2", start, end, dur, 'I41', 'ztf')


#Table Creation
data_table = precovis.tableCreation(merged_healpixels)

#Slider Creation
date_range_slider = precovis.mjdSliderCreation(start,end)

#Download Button Creation
download_button = precovis.Button(label="Download", button_type="default")

#White Space
dumdiv = precovis.Div(text='',width=10)

#White Space 2
dumdiv2 = precovis.Div(text='',width=10)

#Indicator Div for Patch Tapping
indicator_div = precovis.Div(text="no")




#Dropdown Creations
from bokeh.models import Select
obs_select = Select(value='I41', title='Observatory Selection', options=list(np.unique(precovis.frames['obscode'])))
dataset_id_select = Select(value='ztf', title='Dataset Selection', options=list(np.unique(precovis.frames['dataset_id'])))
#asteroid_select = Select(value='2020 AV2', title='Asteroid Selection', options=list()))

#Enter Button Creation
change_button = precovis.Button(label="Change Plot", button_type="default")



#Plot Updating (Main)
plot_callback = precovis.CustomJS(args=dict(all_sources=precovis.all_sources), code="""
    
    const closestIndex = (num, arr) => {
       let curr = arr[0], diff = Math.abs(num - curr);
       let index = 0;
       for (let val = 0; val < arr.length; val++) {
          let newdiff = Math.abs(num - arr[val]);
          if (newdiff < diff) {
             diff = newdiff;
             curr = arr[val];
             index = val;
          };
       };
       return index;
    };

    var data = [];
    var ref = [];
    for (var ii = 0; ii < all_sources.length; ii=ii+2){
        //console.log("ii"+ii)
        data[ii] = all_sources[ii].data;
        ref[ii] = all_sources[ii+1].data;
        

        const index_range = [closestIndex(cb_obj.value[0], ref[ii]["time"]), closestIndex(cb_obj.value[1], ref[ii]["time"])+1];
        
        const columns = Object.keys(data[ii])
        console.log(columns)
        //console.log("Before slider column lengths " + ii + ": " + data[ii]["ra"].length + ", " + data[ii]["dec"].length + ", " + data[ii]["healpixel"].length+ ", " + data[ii]["time"].length);

        for (var c = 0; c < columns.length; c++){
            data[ii][columns[c]] = ref[ii][columns[c]].slice(index_range[0], index_range[1]);
        }
        
        //data[ii]["index"] = ref[ii]["index"].slice(index_range[0], index_range[1]);
        //data[ii]["dec"] = ref[ii]["dec"].slice(index_range[0], index_range[1]);
        //data[ii]["ra"] = ref[ii]["ra"].slice(index_range[0], index_range[1]);
        //data[ii]["healpixel"] = ref[ii]["healpixel"].slice(index_range[0], index_range[1]);
        //data[ii]["time"] = ref[ii]["time"].slice(index_range[0], index_range[1]);
        
        //console.log("After slider column lengths " + ii + ": " + data[ii]["ra"].length + ", " + data[ii]["dec"].length + ", " + data[ii]["healpixel"].length+ ", " + data[ii]["time"].length);

        all_sources[ii].change.emit();
    };

    """)

  

table_callback = precovis.CustomJS(args=dict(source=precovis.table_source, ref_source=precovis.table_ref_source), code="""
   
    const closestIndex = (num, arr) => {
       let curr = arr[0], diff = Math.abs(num - curr);
       let index = 0;
       for (let val = 0; val < arr.length; val++) {
          let newdiff = Math.abs(num - arr[val]);
          if (newdiff < diff) {
             diff = newdiff;
             curr = arr[val];
             index = val;
          };
       };
       return index;
    };
    
    const data = source.data;
    const ref = ref_source.data;

    //console.log("Before slider column lengths: " + data["healpixel"].length + ", " + data["exposure_mjd_start"].length + ", " + data["exposure_mjd_mid"].length + ", " + data["exposure_duration"].length + ", " + data["id"].length + ", " + data["dataset_id"].length + ", " + data["data_uri"].length + ", " + data["obscode"].length + ", " + data["filter"].length);
    
    const index_range = [closestIndex(cb_obj.value[0], ref["exposure_mjd_mid"]), closestIndex(cb_obj.value[1], ref["exposure_mjd_mid"])+1];
    const columns = Object.keys(data)
    
    for (var c = 0; c < columns.length; c++){
        data[columns[c]] = ref[columns[c]].slice(index_range[0], index_range[1]);
    }
    
    //data["healpixel"] = ref["healpixel"].slice(index_range[0], index_range[1]);
    //data["exposure_mjd_start"] = ref["exposure_mjd_start"].slice(index_range[0], index_range[1]);
    //data["exposure_mjd_mid"] = ref["exposure_mjd_mid"].slice(index_range[0], index_range[1]);
    //data["exposure_duration"] = ref["exposure_duration"].slice(index_range[0], index_range[1]);
    //data["id"] = ref["id"].slice(index_range[0], index_range[1]);
    //data["dataset_id"] = ref["dataset_id"].slice(index_range[0], index_range[1]);
    //data["exposure_id"] = ref["exposure_id"].slice(index_range[0], index_range[1]);
    //data["data_uri"] = ref["data_uri"].slice(index_range[0], index_range[1]);
    //data["obscode"] = ref["obscode"].slice(index_range[0], index_range[1]);
    //data["filter"] = ref["filter"].slice(index_range[0], index_range[1]);
    //data["index"] = ref["index"].slice(index_range[0], index_range[1]);
    //data["filter"] = ref["filter"].slice(index_range[0], index_range[1]);
    //data["data_offset"] = ref["data_offset"].slice(index_range[0], index_range[1]);
    //data["data_length"] = ref["data_length"].slice(index_range[0], index_range[1]);
    //data["ra"] = ref["ra"].slice(index_range[0], index_range[1]);
    //data["dec"] = ref["dec"].slice(index_range[0], index_range[1]);
    //data["time"] = ref["time"].slice(index_range[0], index_range[1]);
    
    //console.log("After slider column lengths: " + data["healpixel"].length + ", " + data["exposure_mjd_start"].length + ", " + data["exposure_mjd_mid"].length + ", " + data["exposure_duration"].length + ", " + data["id"].length + ", " + data["dataset_id"].length + ", " + data["data_uri"].length + ", " + data["obscode"].length + ", " + data["filter"].length);
    
    source.change.emit();
    """)


mouse_click_callback = precovis.CustomJS(args=dict(obs_plot_xrange=observation_plot.x_range, obs_plot_yrange=observation_plot.y_range), code="""
    
    var event_x = cb_obj.x
    var event_y = cb_obj.y
    var event_sx = cb_obj.sx
    var event_sy = cb_obj.sy
    
    //console.log("Event x: " + event_x)
    //console.log("X Range Start:" + obs_plot_xrange.start)
    //console.log("X Range:" + obs_plot_xrange)
    
    obs_plot_xrange.start = event_x - 5
    obs_plot_xrange.end = event_x + 5
    obs_plot_yrange.start = event_y - 5
    obs_plot_yrange.end = event_y + 5
    
    obs_plot_xrange.change.emit()
    obs_plot_yrange.change.emit()
    """)


download_button_javaScript= """
        var data = source.data;
        var nrows = source.get_length();
        const columns = Object.keys(data);
        
        var out = "";
        
        for (var c = 0; c < columns.length-1; c++){
            out += columns[c] + ",";
        }
        out = out + columns[columns.length-1] + "\\n"
        
        for (var i = 0; i < nrows; i++) {
            for (var d = 0; d < columns.length-1; d++){
                out += data[columns[d]][i] + ",";
            }
            out += data[columns[columns.length-1]][i] + "\\n"; //This is where the error is
        }
        
        //var out = "healpixel, exposure_mjd_start, exposure_mjd_mjd, exposure_duration, id, dataset_id, exposure_id, data_uri, obscode, filter, data_offset, data_length, ra, dec, time\\n";
        //for (var i = 0; i < nrows; i++) {
        //    out += data['healpixel'][i] + "," + data['exposure_mjd_start'][i] + "," + data['exposure_mjd_mid'][i] + "," + data['exposure_duration'][i] + "," + data['id'][i] + "," + data['dataset_id'][i] + "," + data['exposure_id'][i] + "," + data['data_uri'][i] + "," + data['obscode'][i] + "," + data['filter'][i] +  + "," + data['data_offset'][i]  + "," + data['data_length'][i]  + "," + data['ra'][i]  + "," + data['dec'][i]  + "," + data['time'][i] + "\\n";
        //}
        
        var file = new Blob([out], {type: 'text/plain'});
        var elem = window.document.createElement('a');
        elem.href = window.URL.createObjectURL(file);
        elem.download = 'subsection_orbit_path_data.csv';
        document.body.appendChild(elem);
        elem.click();
        document.body.removeChild(elem);
        """


patch_indicator_callback = precovis.CustomJS(args=dict(patches=precovis.pixel_sources, div=indicator_div), code="""
console.log(cb_obj.indices)
console.log(cds.selected.indices)  
// note that these are the same. cds.selected is the object that this callback is set for,
// so you could use either one of these to refer to the data in your data source (below). 
// if the indices array is empty, user clicked somewhere not on a patch

for (var i = 0; i < pixel_sources.length; i++){
    if (cb_obj.indices.length == 0) {
        div.text = "no"
    }
    else {
        div.text = "yes: "+cds.data.name[cb_obj.indices]
    }
};
""")



orbit_plot.js_on_event(precovis.DoubleTap, mouse_click_callback)

date_range_slider.js_on_change('value_throttled', plot_callback)
date_range_slider.js_on_change('value_throttled', table_callback)

download_button.js_on_event(precovis.ButtonClick, precovis.CustomJS(args=dict(source=precovis.table_source), code=download_button_javaScript))

for pixel_source in precovis.pixel_sources:    
    pixel_source.selected.js_on_change('indices', patch_indicator_callback)




#Add everything to curdoc()
'''
row1 = precovis.Row(orbit_plot,observation_plot)
row2 = precovis.Row(date_range_slider, dumdiv, obs_select, dumdiv2, button)
layout = precovis.Column(row1, row2, data_table)
'''
select_col = precovis.Column(date_range_slider, precovis.Row(obs_select, dumdiv, dataset_id_select), change_button)
row1 = precovis.Row(orbit_plot,select_col)
layout = precovis.Column(row1, observation_plot, dumdiv2, data_table, download_button)
asteroid_doc.add_root(layout)

#precovis.save(asteroid_doc, "slider+table_v05.html")
#Don't touch slider+table.html since that is on the public drive now

end_time = time.time()
total_time = end_time - start_time
print("Time taken to run script:",total_time) #Seems to currently be around 120 seconds (half of them due to sorting dataset_id and obscode)

precovis.save(asteroid_doc, "precovisdoc_v001_001.html")